# Phase Unwrapping

> phase unwrapping

In [ ]:
#| default_exp pu

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import math
import numpy as np
import tempfile
from pathlib import Path
import moraine as mr

In [ ]:
#| export
def gamma_mcf_pt(
    pc_x:np.ndarray, # x coordinate, shape of (N,)
    pc_y:np.ndarray, # y coordinate, shape of (N,)
    ph:np.ndarray, # wrapped phase, shape of (N,)
    ph_weight:np.ndarray=None, # point weight, shape of (N,), optional
    ref_point:int=1, # reference point, the first point by default
) -> np.ndarray: # unwrapped phase, shape of (N,)
    pc_x = pc_x.astype(np.int32)
    pc_y = pc_y.astype(np.int32)
    pc_xy = np.stack((pc_x,pc_y),axis=-1)
    ph = ph.astype(np.complex64)
    if ph_weight is not None: ph_weight = ph_weight.astype(np.float32)

    with tempfile.TemporaryDirectory() as tmpdir_str:
        tmp_path = Path(tmpdir_str)
        pt_path = temp_dir/'pt'
        ph_path = temp_dir/'ph'
        unwrap_ph_path = temp_dir/'unwrap_ph'
        mr.write_gamma
        pg.write_point_list(pt_g(ptx,pty),pt_path,dtype='int')
        pg.write_point_data(ph,pt_path,ph_path,dtype='fcomplex')
        if ph_weight is None:
            ph_weight_path = '-'
        else:
            ph_weight_path = temp_dir/'ph_weight'
            ph_wieght = ph_weight.astype(np.float32)
            pg.write_point_data(ph_weight,pt_path,ph_weight_path,dtype='float')

        pg.mcf_pt(pt_path,'-',ph_path,'-',ph_weight_path,'-',unwrap_ph_path,xps,yps,ref_point,1)

        unwrap_ph = pg.read_point_data(unwrap_ph_path,pt_path,dtype='float')


In [ ]:
def mcf_pt(ptx,pty,xps,yps,ph,ph_weight=None,ref_point=None):
    '''
    a simple wrapper for mcf_pt phase unwrapping in GAMMA software.
    
    Parameters
    ----------
    ptx: ndarray
        point target east coordinate index (int).
    pty: ndarray
        point target north coordinate index (int).
    xps: float
        East coordinate pixel spacing.
    yps: float
        North coordinate pixel spacing.
    ph: (M,N) ndarray
        Complex unwrapped interferograms. M intergerograms and 
        each has N points.
    ph_weight: (M,N) ndarray (optional)
        Unwrapping weight for each points.
    ref_point: int (optional)
        Reference points.
    '''

    if ref_point is None:
        ref_point = '-'
    ph = ph.astype(np.complex64)
    temp_dir = Path.cwd()/unique_str(ph)
    temp_dir.mkdir()
    pt_path = temp_dir/'pt'
    ph_path = temp_dir/'ph'
    unwrap_ph_path = temp_dir/'unwrap_ph'
    pg.write_point_list(pt_g(ptx,pty),pt_path,dtype='int')
    pg.write_point_data(ph,pt_path,ph_path,dtype='fcomplex')
    if ph_weight is None:
        ph_weight_path = '-'
    else:
        ph_weight_path = temp_dir/'ph_weight'
        ph_wieght = ph_weight.astype(np.float32)
        pg.write_point_data(ph_weight,pt_path,ph_weight_path,dtype='float')

    pg.mcf_pt(pt_path,'-',ph_path,'-',ph_weight_path,'-',unwrap_ph_path,xps,yps,ref_point,1)

    unwrap_ph = pg.read_point_data(unwrap_ph_path,pt_path,dtype='float')
    shutil.rmtree(temp_dir)
    
    return unwrap_ph

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()